### This script contains the following points:
#### 1. Importing libraries, defining project path, and importing dataset 'orders_products_merged_2.pkl'
#### 2. Finding the aggregated mean of the 'order_number' column grouped by 'department_id'
#### 3. Analyzing the aggregated mean of the 'order_number' column grouped by 'department_id' 
#### 4. Creating flag 'loyalty_flag' based on 'max_order' (grouped by 'user_id')
#### 5. Analyze column 'prices' (with descriptive statistics) grouped by column 'loyalty_flag'
#### 6. Creating flag 'spend_flag' based on 'avg_price' (grouped by 'user_id')
#### 7. Creating flag 'order_freq_flag' based on 'median_days_since_last_order' (grouped by 'user_id')
#### 8. Exporing dataframe as 'orders_products_merged_2.pkl'

## Step 1. Importing Libraries, Defining Project Path, and Importing Datasets

In [1]:
# Importing pandas, numpy, and os
import pandas as pd
import numpy as np
import os

In [2]:
# Defining project folder path
path = r'C:\Users\prena\03-2023 Instacart Basket Analysis'

In [3]:
# Importing orders_products_combined.pkl dataset
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_2.pkl'))

## Step 2 - 3. Find the aggregated mean of the "order_number" column grouped by "department_id" for dataframe 'ords_prods_merge'

In [4]:
# Get the mean of 'order_number' by 'department_id'
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


This results above compared to the results we got previously (when finding the mean on a subset of data) has more departments represented. Department_id 1 through 21 are shown here when running this code for the entire dataframe vs. only 8 department_id's showing when running this code for our 1,000,000 subset.

In addition, the average number of order numbers are all slightly different (+ / -) with this bigger sample size. For example, department_id 4 had an average of 18.8 orders when analyzing the subset, whereas when analyzing the entire dataframe, the same department_id 4 has an average of 17.8 orders. Another example, department_id 17 had an average of 11.3 orders in the subset, but with the entire dataframe, department_id 17 has an average of 15.7 orders. 

## Step 4a. Create a loyalty flag for existing customers - Create column to hold max value of "order_number" by customer¶

In [5]:
# New column 'max_order' created (grouped by user_id)
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

## Step 4b. Create a loyalty flag for existing customers - Create a column to set flag

In [6]:
# New column 'loyalty_flag' created with the following criteria...
# 'Loyal customer' defined by customers with a max order of greater than 40
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [7]:
# 'Regular customer' defined by customers with a max order of less than or equal to 40 and greater than 10
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [8]:
# 'New customer' defined by customers with a max order of less than or equal to 10
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
# Frequency count for new column 'loyalty_flag'
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15874128
Loyal customer      10282763
New customer         6242841
Name: loyalty_flag, dtype: int64

In [10]:
# Check on new column 'loyalty_flag'
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,...,aisle_id,department_id,prices,_merge_2,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,196,1,0,both,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer
1,2398795,1,2,3,7,15.0,196,1,1,both,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer
2,473747,1,3,3,12,21.0,196,1,1,both,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer
3,2254736,1,4,4,7,29.0,196,1,1,both,...,77,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer
4,431534,1,5,4,15,28.0,196,1,1,both,...,77,7,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer


In [11]:
ords_prods_merge.shape

(32399732, 21)

Looking at the shape of our dataframe, we can see that the columns have increased by 2 (from 19 to 21) to include our two new columns 'max_order' and 'loyalty_flag'.

## Step 5. Analyzing loyalty category with descriptive statistics

In [12]:
ords_prods_merge.groupby('loyalty_flag')['prices'].describe()

,count,mean,std,min,25%,50%,75%,max
loyalty_flag,,,,,,,,
Loyal customer,10282763.0,7.773575,4.206151,1.0,4.2,7.4,11.2,25.0
New customer,6242841.0,7.801206,4.266834,1.0,4.2,7.4,11.3,25.0
Regular customer,15874128.0,7.798262,4.254886,1.0,4.2,7.4,11.3,25.0


Our descriptive statistics for 'prices' for each category of 'loyalty_flag' shows us that the prices of the products that customers are buying are similar regardless of the categry of loyalty they fall under. This makes sense, as customers may be buying similar grocery and household items, and their loyalty is a matter of how often they're utilizing Instacarts services to obtain their groceries (rather than making a grocery store trip themselves). 

## Step 6a. Create a spending flag for existing customers - Create a column to hold average value of "prices" by customer

In [13]:
# New column 'avg_price' created (grouped by user_id)

In [14]:
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.average)

## Step 6b. Create a spending flag for existing customers - Create a column to set flag

In [15]:
# New column 'spend_flag' created with the following criteria...
# 'Low spender' defined by customers with an average price of product less than 10
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spend_flag'] = 'Low spender'

In [16]:
# 'High spender' defined by customers with an average price of product greater than or equal to 10
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spend_flag'] = 'High spender'

In [17]:
# Frequency count for new column 'spend_flag'
ords_prods_merge['spend_flag'].value_counts(dropna = False)

Low spender     32280045
High spender      119687
Name: spend_flag, dtype: int64

In [18]:
# Check on new column 'spend_flag'
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,...,prices,_merge_2,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spend_flag
0,2539329,1,1,2,8,NaN,196,1,0,both,...,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
1,2398795,1,2,3,7,15.0,196,1,1,both,...,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,6.367797,Low spender
2,473747,1,3,3,12,21.0,196,1,1,both,...,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low spender
3,2254736,1,4,4,7,29.0,196,1,1,both,...,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender
4,431534,1,5,4,15,28.0,196,1,1,both,...,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low spender


In [19]:
ords_prods_merge.shape

(32399732, 23)

Looking at the shape of our dataframe, we can see that the columns have increased by 2 (from 21 to 23) to include our two new columns 'avg_price' and 'spend_flag'.

## Step 7a. Create an order frequency flag for existing customers - Create a column to hold median value of "days_since_last_order" by customer

In [20]:
# New column 'median_days_since_last_order' created (grouped by user_id)

In [21]:
ords_prods_merge['median_days_since_last_order'] = ords_prods_merge.groupby(['user_id'])['days_since_last_order'].transform(np.median)

## Step 7b. Create an order frequency flag for existing customers - Create a column to set flag

In [22]:
# New column 'order_freq_flag' created with the following criteria...
# 'Non-frequent customer' defined by customers with a median days since last order as greater than 20 days
ords_prods_merge.loc[ords_prods_merge['median_days_since_last_order'] > 20, 'order_freq_flag'] = 'Non-frequent customer'

In [23]:
# 'Regular customer' defined by customers with a median days since last order as greater than 10 days and less than or equal to 20 days
ords_prods_merge.loc[(ords_prods_merge['median_days_since_last_order'] > 10) & (ords_prods_merge['median_days_since_last_order'] <= 20), 'order_freq_flag'] = 'Regular customer'

In [24]:
# 'Frequent customer' defined by customers with a median days since last order as less than or equal to 20 days
ords_prods_merge.loc[ords_prods_merge['median_days_since_last_order'] <= 10, 'order_freq_flag'] = 'Frequent customer'

In [25]:
# Frequency count for new column 'order_freq_flag'
ords_prods_merge['order_freq_flag'].value_counts(dropna = False)

Frequent customer        21556644
Regular customer          7207340
Non-frequent customer     3635743
NaN                             5
Name: order_freq_flag, dtype: int64

In [26]:
# Check on new column 'order_freq_flag'
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spend_flag,median_days_since_last_order,order_freq_flag
0,2539329,1,1,2,8,NaN,196,1,0,both,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,both,...,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,both,...,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,both,...,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,both,...,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [27]:
ords_prods_merge.shape

(32399732, 25)

Looking at the shape of our dataframe, we can see that the columns have increased by 2 (from 23 to 25) to include our two new columns 'median_days_since_last_order' and 'order_freq_flag'. Our frequency table shows us that there are five rows that have a missing value for 'order_freq_flag.' Digging into this below:

## Step 7c. Investigating missing values

In [28]:
# Create a new dataframe (df_nan) to look at the rows with missing values for 'order_freq_flag'
df_nan = ords_prods_merge[ords_prods_merge['order_freq_flag'].isnull() == True]

In [29]:
df_nan

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spend_flag,median_days_since_last_order,order_freq_flag
13641263,895835,159838,1,0,17,NaN,10749,3,0,both,...,Mid-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN
17247561,895835,159838,1,0,17,NaN,33401,6,0,both,...,Mid-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN
17618338,895835,159838,1,0,17,NaN,23695,2,0,both,...,Low-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN
24134164,895835,159838,1,0,17,NaN,21334,5,0,both,...,Mid-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN
25875573,895835,159838,1,0,17,NaN,22198,1,0,both,...,Low-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN


Looking at these records, all of them are associated with user_id 159383. Are these the only orders associated with this user_id?

In [30]:
# Looking to see all orders associated with user_id 159838
ords_prods_merge[ords_prods_merge['user_id']==159838]

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spend_flag,median_days_since_last_order,order_freq_flag
13641263,895835,159838,1,0,17,NaN,10749,3,0,both,...,Mid-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN
17247561,895835,159838,1,0,17,NaN,33401,6,0,both,...,Mid-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN
17618338,895835,159838,1,0,17,NaN,23695,2,0,both,...,Low-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN
24134164,895835,159838,1,0,17,NaN,21334,5,0,both,...,Mid-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN
25875573,895835,159838,1,0,17,NaN,22198,1,0,both,...,Low-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN


It looks like these are the only orders associated with customer 159383. As we learned previously, first orders are indicated as "NaN" for field 'days_since_last_order' (since this field can't be calculated for the first order). Therefore, our 'order_freq_flag' can't be calculated, either. Is this the only customer that only has one order in the system?

In [31]:
# Check for max_order = 1 to find other user_id's that only have one order placed
ords_prods_merge[ords_prods_merge['max_order']==1]

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spend_flag,median_days_since_last_order,order_freq_flag
13641263,895835,159838,1,0,17,NaN,10749,3,0,both,...,Mid-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN
17247561,895835,159838,1,0,17,NaN,33401,6,0,both,...,Mid-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN
17618338,895835,159838,1,0,17,NaN,23695,2,0,both,...,Low-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN
24134164,895835,159838,1,0,17,NaN,21334,5,0,both,...,Mid-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN
25875573,895835,159838,1,0,17,NaN,22198,1,0,both,...,Low-range product,Busiest day,Busiest day,Average orders,1,New customer,7.42,Low spender,NaN,NaN


This shows us that customer 159838 is the only customer that has a 'max_order' value of 1 (in other words, the only customer that has placed only one order). Because this is the case, we don't need to take any additional action for the missing values in 'max_order'. Let's take one last look at the max_order frequency table:

In [32]:
ords_prods_merge['max_order'].value_counts(dropna = False)

99    1171076
8      811706
6      811237
9      810057
7      803838
       ...   
97      44949
98      44585
96      40449
2           6
1           5
Name: max_order, Length: 99, dtype: int64

(The '5' listed for value 1 represents customer 159838 above).

# Step 8. Exporting 'df_ords_prods_merged' to 'orders_products_merged_3.pkl'

In [33]:
# Export data to pkl
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_3.pkl'))